In [122]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV

# read data

In [5]:
df_train = pd.read_csv('../datasets/train.csv')
df_test = pd.read_csv('../datasets/test.csv')

In [195]:
def convert_passengerID_2_index(df: pd.DataFrame):

    df = df.set_index("PassengerId")
    return df

def split_carbin(df: pd.DataFrame):

    for key, value in enumerate(['deck', 'num', 'side']):
        df[f'Carbin_{value}'] = df['Cabin'].apply(lambda x: str(x).split("/")[key] if len(str(x).split("/")) >= key+1 else None)

    df = df.drop('Cabin', axis=1)
    return df

In [196]:
df_train_cleansed = (
    df_train.pipe(convert_passengerID_2_index)
    .pipe(split_carbin)
)

In [200]:
df_test_cleansed = (
    df_test.pipe(convert_passengerID_2_index)
    .pipe(split_carbin)
)

# split data into X and y

In [201]:
def split_data_2_x_y(df:pd.DataFrame):

    columns = df.columns 
    X_columns = [i for i in columns if i != 'Transported']
    y_column = 'Transported' if 'Transported' in columns else None 

    if y_column:
        return df[X_columns], df[y_column]
    else:
        return df[X_columns], None

In [202]:
X, y = split_data_2_x_y(df_train_cleansed)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [203]:
X_test, _ = split_data_2_x_y(df_test_cleansed)

In [204]:
X_train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Carbin_deck,Carbin_num,Carbin_side
PassengerId,,,,,,,,,,,,,,
2513_01,Earth,False,TRAPPIST-1e,28.0,False,0.0,55.0,0.0,656.0,0.0,Loree Mathison,nan,None,None
2774_02,Earth,False,TRAPPIST-1e,17.0,False,0.0,1195.0,31.0,0.0,0.0,Crisey Mcbriddley,F,575,P
8862_04,Europa,True,55 Cancri e,28.0,False,0.0,0.0,0.0,0.0,0.0,Alramix Myling,C,329,S
8736_02,Mars,False,TRAPPIST-1e,20.0,False,NaN,2.0,289.0,976.0,0.0,Tros Pota,F,1800,P
0539_02,Europa,True,55 Cancri e,36.0,False,0.0,0.0,0.0,0.0,0.0,Achyon Nalanet,C,18,P


# data pipeline

In [205]:
num_pipeline = Pipeline([
    ("Fill N/A value", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler())
])

cate_pipeline = Pipeline([
    ("Fill N/A value", SimpleImputer(strategy="most_frequent")),
    ("One Hot Encoder", OneHotEncoder(handle_unknown="ignore"))
])

In [33]:
# num_attribs = X_train.select_dtypes(include="float").columns
# cat_attribs = X_train.select_dtypes(include="object").columns

In [206]:
preprocessing = ColumnTransformer([
    ("num", num_pipeline, make_column_selector(dtype_include=np.number)),
    ("cat", cate_pipeline, make_column_selector(dtype_include=object))
])

In [207]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6954 entries, 2513_01 to 7775_01
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    6786 non-null   object 
 1   CryoSleep     6777 non-null   object 
 2   Destination   6815 non-null   object 
 3   Age           6806 non-null   float64
 4   VIP           6792 non-null   object 
 5   RoomService   6828 non-null   float64
 6   FoodCourt     6814 non-null   float64
 7   ShoppingMall  6789 non-null   float64
 8   Spa           6820 non-null   float64
 9   VRDeck        6803 non-null   float64
 10  Name          6795 non-null   object 
 11  Carbin_deck   6954 non-null   object 
 12  Carbin_num    6796 non-null   object 
 13  Carbin_side   6796 non-null   object 
dtypes: float64(6), object(8)
memory usage: 814.9+ KB


# Train the model

In [208]:
rnd_clf = make_pipeline(preprocessing, RandomForestClassifier(random_state=42))
rnd_clf.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x296469d20>),
                                                 ('cat',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('One Hot '
                                                                   'Encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x296469060>)])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])

# Tunning the model

In [209]:
full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("random_forest", RandomForestClassifier(random_state=42))
])

# param_grid = [
#     {'preprocessing__geo__n_clusters': [5, 8, 10],
#     'random_forest__max_features': [4, 6, 8]},
#     {'preprocessing__geo__n_clusters': [10, 15],
#      'random_forest__max_features': [6, 8, 10]}
# ]
param_grid = [
    {
    'random_forest__max_features': [6, 8, 10],
    'random_forest__n_estimators': [50, 100, 200, 500],
    'random_forest__max_depth': [5, 10, None],
    'random_forest__min_samples_split': [2, 5, 10],
    'random_forest__min_samples_leaf': [1, 2, 4],
    # 'random_forest__max_features': ['auto', 'sqrt']
    }
]
grid_search = GridSearchCV(full_pipeline, param_grid, cv=3,
                           scoring="f1", #"roc_auc", 
                           n_jobs=-1)
grid_search.fit(X_train, y_train)

/Users/liuyifu/miniconda3/envs/sklearn-env/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('Fill '
                                                                                          'N/A '
                                                                                          'value',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardize',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x296469d20>),
                                                                        ('cat',
                                                                         Pipeline(steps=[('Fill '
                                                                                          'N/A '
                                                                                          'value',
                                                                                          SimpleImputer(strategy=...
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x296469060>)])),
                                       ('random_forest',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid=[{'random_forest__max_depth': [5, 10, None],
                          'random_forest__max_features': [6, 8, 10],
                          'random_forest__min_samples_leaf': [1, 2, 4],
                          'random_forest__min_samples_split': [2, 5, 10],
                          'random_forest__n_estimators': [50, 100, 200, 500]}],
             scoring='f1')

In [210]:
print('Best hyperparameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

Best hyperparameters: {'random_forest__max_depth': None, 'random_forest__max_features': 8, 'random_forest__min_samples_leaf': 1, 'random_forest__min_samples_split': 2, 'random_forest__n_estimators': 500}
Best score: 0.7791653701659652


In [211]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by='mean_test_score', ascending=False, inplace=True)
cv_res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_random_forest__max_depth,param_random_forest__max_features,param_random_forest__min_samples_leaf,param_random_forest__min_samples_split,param_random_forest__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
255,6.781391,0.083320,1.092751,0.023555,None,8,1,2,500,"{'random_forest__max_depth': None, 'random_for...",0.762456,0.785072,0.789969,0.779165,0.011984,1
222,2.256760,0.032256,0.518174,0.030457,None,6,1,5,200,"{'random_forest__max_depth': None, 'random_for...",0.763717,0.780797,0.792705,0.779073,0.011897,2
219,7.138779,0.206247,1.311602,0.031791,None,6,1,2,500,"{'random_forest__max_depth': None, 'random_for...",0.757848,0.785326,0.790739,0.777971,0.014400,3
218,2.906928,0.118624,0.588130,0.055670,None,6,1,2,200,"{'random_forest__max_depth': None, 'random_for...",0.756635,0.784153,0.792335,0.777708,0.015270,4
223,5.389911,0.073926,1.292544,0.014418,None,6,1,5,500,"{'random_forest__max_depth': None, 'random_for...",0.760714,0.776322,0.794814,0.777284,0.013938,5


# Evaluate the performance

## Evaluate the performance on the training set

In [212]:
y_proba = rnd_clf.predict_proba(X_train)[:, 1]
y_pred = rnd_clf.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 1.0
F1 score: 1.0


In [213]:
y_proba = grid_search.predict_proba(X_train)[:, 1]
y_pred = grid_search.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 1.0
F1 score: 1.0


### Analyse Features

In [214]:
len(grid_search_model["preprocessing"].get_feature_names_out())

12242

In [215]:
grid_search_model = grid_search.best_estimator_
# feature_importances = grid_search_model
feature_importances = grid_search_model["random_forest"].feature_importances_
feature_importances.round(4)

sorted(zip(feature_importances, grid_search_model["preprocessing"].get_feature_names_out()), reverse=True)

[(0.04927741346915821, 'num__Spa'),
 (0.04557209203998364, 'num__VRDeck'),
 (0.041749451682512144, 'num__RoomService'),
 (0.04116516847631946, 'cat__CryoSleep_False'),
 (0.037608897435811216, 'cat__CryoSleep_True'),
 (0.036562419080578196, 'num__Age'),
 (0.036286019386500855, 'num__FoodCourt'),
 (0.033769919378290825, 'num__ShoppingMall'),
 (0.011845850411031977, 'cat__HomePlanet_Earth'),
 (0.011247514541147936, 'cat__HomePlanet_Europa'),
 (0.007350530892214028, 'cat__Carbin_deck_F'),
 (0.006441496232210752, 'cat__Carbin_side_S'),
 (0.006321965516549628, 'cat__Carbin_side_P'),
 (0.006103684253544007, 'cat__HomePlanet_Mars'),
 (0.005964230816573467, 'cat__Carbin_deck_E'),
 (0.005652120174383293, 'cat__Destination_TRAPPIST-1e'),
 (0.005438560605801148, 'cat__Destination_55 Cancri e'),
 (0.005328672256273616, 'cat__Carbin_deck_B'),
 (0.005082850035452586, 'cat__Carbin_deck_G'),
 (0.004932332973918226, 'cat__Carbin_deck_C'),
 (0.0026914024035826537, 'cat__Destination_PSO J318.5-22'),
 (0.0

## Evaluate the performance on the validation set

In [216]:
# randome forest tree
y_proba = rnd_clf.predict_proba(X_val)[:, 1]
y_pred = rnd_clf.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.8616292174962101
F1 score: 0.7908045977011493


In [217]:
# gridsearchCV
y_proba = grid_search.predict_proba(X_val)[:, 1]
y_pred = grid_search.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.8677294241214459
F1 score: 0.789289871944121


# output data

In [220]:
y_test = rnd_clf.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv') # 0.7912

In [221]:
y_test = grid_search.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv') # 0.7877